In [27]:
# 라이브러리

import pandas as pd ; pd.options.display.float_format = '{:.2f}'.format # 소수 둘째자리까지 표시
import numpy as np
import math

import time
from time import sleep
import random
import datetime
from datetime import datetime, timedelta, datetime
import json

from string import ascii_lowercase

import collections
from dateutil.relativedelta import relativedelta

from powernad.API.Campaign import *
from powernad.API.RelKwdStat import *

from urllib.error import HTTPError
import urllib.request

from bs4 import BeautifulSoup
from selenium import webdriver
import psycopg2
import re

from tqdm.notebook import tqdm

import requests; from urllib.parse import urlparse
import seaborn as sns; sns.set(style='darkgrid', font='NanumGothic', font_scale=1.2)
import matplotlib; import matplotlib.pyplot as plt
import squarify

In [24]:
# Azure DB용 라이브러리

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import skipgrams
from sqlalchemy import create_engine
import psycopg2
import pandas as pd
import datetime

In [28]:
def load_dataframe():
    """
    azure에 있는 'book' table을 불러오는 함수입니다.
    불러온 테이블을 pandas DataFrame 형태로 반환합니다. 

    ...
    Parameters
    ----------
   
    Return
    pandas.DataFrame
    
    
    """
    connection = psycopg2.connect(
    "host=kedu.postgres.database.azure.com dbname=postgres user=keduadmin password=1qaz2wsx! port=5432")
    cur = connection.cursor()

    query = """ select * from kedu5.book_edited """ 

    cur.execute(query)


    aa = cur.fetchall()

    df = pd.DataFrame(aa)

    #dataframe 반환전에 컬럼을 설정해줍니다. 
    df.columns = ['index','rank', 'title', 'category', 'year', 'month', 'source']

    
    return df

In [32]:
df_origin = load_dataframe()
df_origin.tail(10)

,index,rank,title,category,year,month,source
243936,185638,547,삶으로서의 일,자기계발,2022,5,알라딘
243937,185639,548,스토리씽킹,자기계발,2022,5,알라딘
243938,185640,549,절대 배신하지 않는 공부의 기술,자기계발,2022,5,알라딘
243939,185641,550,1페이지 공부법,자기계발,2022,5,알라딘
243940,185642,551,세바시 인생질문 1 : 나는 누구인가,자기계발,2022,5,알라딘
243941,185643,552,IT 좀 아는 사람,자기계발,2022,5,알라딘
243942,185644,553,빨강 머리 앤이 3년 후 나에게 Q & A a day (램스킨 리미티드 에디션),자기계발,2022,5,알라딘
243943,185645,554,습관의 디테일,자기계발,2022,5,알라딘
243944,185647,555,빨강 머리 앤이 10년 후 나에게 : Q&A a day,자기계발,2022,5,알라딘
243945,185648,556,적을 만들지 않는 대화법,자기계발,2022,5,알라딘


# 구간날짜 변수 생성

In [30]:
"""
날짜 구간 정보가 들어있는 변수들을 생성합니다.
이 변수들을 활용해 날짜 구간을 기준으로 매일 데이터를 갱신합니다.
...
----------------------------------
today = 현재
today = 오늘자 년월일 문자열
yesterday = 어제 
time_week = 7일 전
time_month = 1개월 전
time_year = 1년 전
time_two_years = 2년 전
time_three_years = 3년 전
"날짜구간 변수명"_index = 날짜구간 인덱스
dt_"날짜구간" = 날짜구간 인덱스를 리스트 형태로 
"""

# 오늘
toda = datetime.now()
today = str(datetime.now().date()) # 


# 1일 전
yesterday = toda - relativedelta(days=1)
yesterday = yesterday.strftime('%Y-%m-%d')
yesterday = str(yesterday)


# 1주
time_week = toda - relativedelta(weeks=1)
time_week = time_week.strftime('%Y-%m-%d')
time_week = str(time_week)
# 최근 1주 리스트
week_index = pd.date_range(start=time_week, end= yesterday)
dt_week = week_index.strftime("%Y-%m-%d").tolist()


# 1개월
time_month = toda - relativedelta(months=1)
time_month = time_month.strftime('%Y-%m-%d')
time_month= str(time_month)
# 최근 1개월 리스트
month_index = pd.date_range(start=time_month, end= yesterday)
dt_month = month_index.strftime("%Y-%m-%d").tolist()


# 1년
time_year = toda - relativedelta(years=1)
time_year = time_year.strftime('%Y-%m-%d')
time_year= str(time_year)
# 최근 1년 리스트
year_index = pd.date_range(start=time_year, end= yesterday)
dt_year = year_index.strftime("%Y-%m-%d").tolist()


# 2년
time_two_years = toda - relativedelta(years=2)
time_two_years = time_two_years.strftime('%Y-%m-%d')
time_two_years = str(time_two_years)
# 최근 2년 리스트
two_years_index = pd.date_range(start=time_two_years, end= yesterday)
dt_two_years = two_years_index.strftime("%Y-%m-%d").tolist()

# 3년
time_three_years = toda - relativedelta(years=3)
time_three_years = time_three_years.strftime('%Y-%m-%d')
time_three_years = str(time_three_years)
# 최근 3년 리스트
three_years_index = pd.date_range(start=time_three_years, end= yesterday)
dt_three_years = three_years_index.strftime("%Y-%m-%d").tolist()

In [33]:
# 	rank	keyword	year	month	day	ymd
df_origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243946 entries, 0 to 243945
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   index     243946 non-null  int64 
 1   rank      243946 non-null  int64 
 2   title     243946 non-null  object
 3   category  243946 non-null  object
 4   year      243946 non-null  int64 
 5   month     243946 non-null  int64 
 6   source    243946 non-null  object
dtypes: int64(4), object(3)
memory usage: 13.0+ MB


In [58]:
df_origin.iloc[:,1:2]
df_origin.iloc[:,3:]
temp = df_origin.drop_duplicates(['title'])
zxc = temp[temp['year'] == 2022]
zxc = zxc[zxc['month'] >= 5]
zxc

,index,rank,title,category,year,month,source
234613,229660,2,천관사복 1~5 특별판 화성 세트 - 전5권,라이트노벨,2022,5,알라딘
234614,229661,3,이방인 3 북케이스 세트 (아크릴스탠드 포함 VER.),라이트노벨,2022,5,알라딘
234615,229662,4,언제나 밤인 세계,라이트노벨,2022,5,알라딘
234616,229663,5,행성 1,라이트노벨,2022,5,알라딘
234617,229664,6,행성 2,라이트노벨,2022,5,알라딘
...,...,...,...,...,...,...,...
243918,185620,529,어른의 문해력,자기계발,2022,5,알라딘
243919,185621,530,부자의 독서법,자기계발,2022,5,알라딘
243920,185622,531,이것이 메타버스 마케팅이다,자기계발,2022,5,알라딘
243923,185625,534,일본어 출판 번역 수업,자기계발,2022,5,알라딘


In [ ]:
# df_origin
"""
네이버 데이터랩 쇼핑인사이트 기준 일자별 패션의류 카테고리 검색랭킹 500위에 포함된 키워드 정보가 있는 데이터프레임입니다.
데이터 시작일 = 2017-08-01
...
rank = 일자별 네이버 데이터랩 검색어 랭킹 (1 ~ 500)
keyword = 검색어 이름
year = 검색어 랭크인한 년도 정보
month = 검색어 랭크인한 월 정보
day = 검색어 랭크인한 일자 정보
ymd = 검색어 랭크인한 당일 년월일 정보
"""
df_origin = pd.read_csv('C:/project/CODE/keyword/data/fashion/naver_top_500/' + yesterday + '_naver_fashion_updated.csv', header = None, encoding='utf-8', index_col=0)
df_origin.columns = ['rank', 'keyword', 'year', 'month', 'day', 'ymd']
df_origin = df_origin.sort_values(by=['ymd', 'rank'])
df_origin = df_origin.reset_index(drop=True)
df_origin